In [17]:
import os
import numpy as np
import pandas as pd

In [18]:
excel_data_filename = r'Basis State Industry List 071621.xlsx'
excel_industries_filename = r'Basis State Industries 210701.xlsx'
excel_industries_range = r'B6:B105'

In [19]:

 
training_df = pd.read_excel(open('Basis State Industry List 071621.xlsx', 'rb'),
              sheet_name='Training List')
training_df

,Organization - ID,Organization - Name,Organization - Website,Organization - CB Description,Organization - Company Type,Organization - CB Industries,Organization - Employee Range,Organization - New Industry,Keyword,MetaKeyword,MetaDescription
0,440,Aston Martin Lagonda,https://www.astonmartin.comen-us/,Aston Martin Lagonda Global Holdings manufactu...,Acquirer,"Automotive, Leisure, Manufacturing, Product De...",1001-5000,Automotive - Car Manufacturing,NaN,NaN,NaN
1,898,Fiat Chrysler Auto,https://www.fcagroup.comen-US/Pages/home.aspx,"Fiat Chrysler Automobiles designs, engineers, ...",Acquirer,"Automotive, Industrial Manufacturing, Manufact...",10001+,Automotive - Car Manufacturing,NaN,NaN,NaN
2,930,Ford Motor Company,https://www.ford.com,"Ford Motor Company, an automotive company, dev...",Acquirer,"Apps, Automotive, Manufacturing, Transportation",10001+,Automotive - Car Manufacturing,access,NaN,NaN
3,965,General Motors Company,https://www.gm.com,General Motors is a Detroit-based mutinational...,Acquirer,"Automotive, Enterprise Software, Manufacturing...",10001+,Automotive - Car Manufacturing,access,NaN,NaN
4,296,AARP,http://www.aarp.org,NaN,Acquirer,NaN,NaN,"Civic, Non-Profit and Membership Groups","benefits, website, leaving, provider, cancel, ...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2164,48844,Healthtrust Workforce Solutions,https://healthtrustjobs.com/,NaN,Acquirer,NaN,NaN,NaN,NaN,NaN,NaN
2165,48845,Signature Consultants,https://www.sigconsult.com/,NaN,Acquirer,NaN,NaN,NaN,NaN,NaN,NaN
2166,48846,Acro Service Corporation,https://www.acrocorp.com/,NaN,Acquirer,NaN,NaN,NaN,NaN,NaN,NaN
2167,48847,DISYS,https://www.disys.com/,NaN,Acquirer,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:

training_df.columns=['org_id','org_name','org_website','org_cb_description','org_company_type','org_cb_industry','org_company_size','org_new_industry','keyword','metakeyword','metadesc']
drop_col=['org_id','org_name','org_website','org_company_type','org_cb_industry','org_company_size']
training_df.drop(columns=drop_col,axis=1, inplace=True)
training_df


,org_cb_description,org_new_industry,keyword,metakeyword,metadesc
0,Aston Martin Lagonda Global Holdings manufactu...,Automotive - Car Manufacturing,NaN,NaN,NaN
1,"Fiat Chrysler Automobiles designs, engineers, ...",Automotive - Car Manufacturing,NaN,NaN,NaN
2,"Ford Motor Company, an automotive company, dev...",Automotive - Car Manufacturing,access,NaN,NaN
3,General Motors is a Detroit-based mutinational...,Automotive - Car Manufacturing,access,NaN,NaN
4,NaN,"Civic, Non-Profit and Membership Groups","benefits, website, leaving, provider, cancel, ...",NaN,NaN
...,...,...,...,...,...
2164,NaN,NaN,NaN,NaN,NaN
2165,NaN,NaN,NaN,NaN,NaN
2166,NaN,NaN,NaN,NaN,NaN
2167,NaN,NaN,NaN,NaN,NaN


In [21]:
training_df = training_df[training_df['org_new_industry'].notna()]
training_df

,org_cb_description,org_new_industry,keyword,metakeyword,metadesc
0,Aston Martin Lagonda Global Holdings manufactu...,Automotive - Car Manufacturing,NaN,NaN,NaN
1,"Fiat Chrysler Automobiles designs, engineers, ...",Automotive - Car Manufacturing,NaN,NaN,NaN
2,"Ford Motor Company, an automotive company, dev...",Automotive - Car Manufacturing,access,NaN,NaN
3,General Motors is a Detroit-based mutinational...,Automotive - Car Manufacturing,access,NaN,NaN
4,NaN,"Civic, Non-Profit and Membership Groups","benefits, website, leaving, provider, cancel, ...",NaN,NaN
...,...,...,...,...,...
2120,JetBlue airways to bring humanity back to air ...,Transportation - Airline,NaN,NaN,NaN
2121,Southwest Airlines (NYSE: LUV) continues to di...,Transportation - Airline,NaN,NaN,NaN
2122,United Airlines and United Express operate app...,Transportation - Airline,NaN,NaN,NaN
2123,FedEx Corporation provides customers and busin...,Transportation - Shipping,NaN,NaN,NaN


In [25]:
training_df['description_combined'] = training_df[['org_cb_description', 'keyword', 'metakeyword','metadesc']].fillna('').agg(' '.join, axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [44]:
training_df.dtypes

org_cb_description      object
org_new_industry        object
keyword                 object
metakeyword             object
metadesc                object
description             object
description_combined    object
dtype: object

In [51]:
train_df=training_df.loc[:,['org_new_industry','description_combined']]
train_df

,org_new_industry,description_combined
0,Automotive - Car Manufacturing,Aston Martin Lagonda Global Holdings manufactu...
1,Automotive - Car Manufacturing,"Fiat Chrysler Automobiles designs, engineers, ..."
2,Automotive - Car Manufacturing,"Ford Motor Company, an automotive company, dev..."
3,Automotive - Car Manufacturing,General Motors is a Detroit-based mutinational...
4,"Civic, Non-Profit and Membership Groups","benefits, website, leaving, provider, cancel,..."
...,...,...
2120,Transportation - Airline,JetBlue airways to bring humanity back to air ...
2121,Transportation - Airline,Southwest Airlines (NYSE: LUV) continues to di...
2122,Transportation - Airline,United Airlines and United Express operate app...
2123,Transportation - Shipping,FedEx Corporation provides customers and busin...


In [29]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-08-06 04:01:04--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-08-06 04:01:05--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               setup Colab for PySpark 3.1.2 and Spark NLP 3.1.3
-                   100%[==

In [30]:
import sparknlp
spark = sparknlp.start() 
# sparknlp.start(gpu=True) >> for training on GPU
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline


Text Classification with Classifier with SparkNLP

In [56]:
document = DocumentAssembler()\
    .setInputCol("description_combined")\
    .setOutputCol("document")

In [57]:
use = UniversalSentenceEncoder.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [58]:
classsifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("org_new_industry")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

In [59]:
use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

In [53]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()

train_sparkdf=spark.createDataFrame(train_df) 

In [60]:
use_pipelineModel = use_clf_pipeline.fit(train_sparkdf)

In [61]:
!cat /root/annotator_logs/ClassifierDLApproach_c570d0becc72.log

Training started - epochs: 5 - learning_rate: 0.005 - batch_size: 64 - training_examples: 2125 - classes: 90
Epoch 0/5 - 0.66s - loss: 152.42564 - acc: 0.057291668 - batches: 34
Epoch 1/5 - 0.44s - loss: 151.63655 - acc: 0.068655305 - batches: 34
Epoch 2/5 - 0.45s - loss: 151.63655 - acc: 0.068655305 - batches: 34
Epoch 3/5 - 0.40s - loss: 151.63655 - acc: 0.068655305 - batches: 34
Epoch 4/5 - 0.40s - loss: 151.63655 - acc: 0.068655305 - batches: 34


Bert Sentence Embedding 

In [ ]:
document = DocumentAssembler()\
    .setInputCol("description_combined")\
    .setOutputCol("document")

In [62]:
bert_sent=BertSentenceEmbeddings.pretrained("sent_small_bert_L8_512")\
.setInputCols(["document"])\
.setOutputCol("sentence_embeddings")

sent_small_bert_L8_512 download started this may take some time.
Approximate size to download 149.1 MB
[OK!]


In [66]:
classsifierd2 = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("org_new_industry")\
  .setMaxEpochs(20)\
  .setEnableOutputLogs(True)

In [67]:
bert_sent_clf_pipeline=Pipeline(
    stages=
    [document,bert_sent,classsifierd2])

In [68]:
bert_sentPiplelineModel=bert_sent_clf_pipeline.fit(train_sparkdf)

In [69]:
!cat /root/annotator_logs/ClassifierDLApproach_5546c333482b.log

Training started - epochs: 20 - learning_rate: 0.005 - batch_size: 64 - training_examples: 2125 - classes: 90
Epoch 0/20 - 0.53s - loss: 151.60966 - acc: 0.06767191 - batches: 34
Epoch 1/20 - 0.39s - loss: 151.98505 - acc: 0.07149621 - batches: 34
Epoch 2/20 - 0.40s - loss: 151.63652 - acc: 0.06581439 - batches: 34
Epoch 3/20 - 0.39s - loss: 151.63655 - acc: 0.06581439 - batches: 34
Epoch 4/20 - 0.40s - loss: 151.63655 - acc: 0.06581439 - batches: 34
Epoch 5/20 - 0.40s - loss: 151.63655 - acc: 0.06581439 - batches: 34
Epoch 6/20 - 0.39s - loss: 151.63655 - acc: 0.06581439 - batches: 34
Epoch 7/20 - 0.39s - loss: 151.63655 - acc: 0.06581439 - batches: 34
Epoch 8/20 - 0.39s - loss: 151.63655 - acc: 0.06581439 - batches: 34
Epoch 9/20 - 0.38s - loss: 151.63655 - acc: 0.06581439 - batches: 34
Epoch 10/20 - 0.38s - loss: 151.63655 - acc: 0.06581439 - batches: 34
Epoch 11/20 - 0.39s - loss: 151.63655 - acc: 0.06581439 - batches: 34
Epoch 12/20 - 0.39s - loss: 151.63655 - acc: 0.06581439 - ba

Using Glove Embeddings

In [81]:
document_assembler = DocumentAssembler() \
    .setInputCol("description_combined") \
    .setOutputCol("document")
    
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")
      
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
    .setInputCols("normalized")\
    .setOutputCol("cleanTokens")\
    .setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma")




lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [84]:
glove_embeddings = WordEmbeddingsModel().pretrained() \
      .setInputCols(["document",'lemma'])\
      .setOutputCol("embeddings")\
      .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

classsifierd3 = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("org_new_industry")\
      .setMaxEpochs(3)\
      .setEnableOutputLogs(True)
      #.setOutputLogsPath('logs')

clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            embeddingsSentence,
            classsifierd3])

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [85]:
clf_pipelineModel = clf_pipeline.fit(train_sparkdf)

In [86]:
!cat /root/annotator_logs/ClassifierDLApproach_ad793ec19836.log

Training started - epochs: 3 - learning_rate: 0.005 - batch_size: 64 - training_examples: 2125 - classes: 90
Epoch 0/3 - 0.59s - loss: 150.31068 - acc: 0.053430945 - batches: 34
Epoch 1/3 - 0.53s - loss: 150.63655 - acc: 0.058165792 - batches: 34
Epoch 2/3 - 0.41s - loss: 150.63657 - acc: 0.058165792 - batches: 34
